In [94]:
import numpy as np
import pandas as pd
from collections import namedtuple
Portfolio = namedtuple('Portfolio', 'type code price delta gamma vega theta')
w1 = -1000
stock = Portfolio('stock', '0700', 472, 1, 0, 0, 0)
option1 = Portfolio('call_option', 'TCH220225C500000', 8.220, 0.293, 0.007, 0.410, -0.343)
option2 = Portfolio('call_option', 'TCH220225C510000', 6.060, 0.229, 0.006, 0.360, -0.305)
option3 = Portfolio('call_option', 'TCH220225C520000', 4.450, 0.176, 0.005, 0.308, -0.261)
pd.DataFrame([stock, option1, option2, option3])

,type,code,price,delta,gamma,vega,theta
0,stock,0700,472.00,1.000,0.000,0.000,0.000
1,call_option,TCH220225C500000,8.22,0.293,0.007,0.410,-0.343
2,call_option,TCH220225C510000,6.06,0.229,0.006,0.360,-0.305
3,call_option,TCH220225C520000,4.45,0.176,0.005,0.308,-0.261


In [95]:
# greek neutralization -- gamma and vega
greeks = np.array([[option2.gamma, option3.gamma], [option2.vega, option3.vega]])
portfolio_greeks = [[option1.gamma * abs(w1)], [option1.vega * abs(w1)]]
inv = np.linalg.inv(np.round(greeks, 2))  # We need to round otherwise we can end up with a non-invertible matrix
# position on option 2 and 3 to be gamma and vega neutral
w = np.dot(inv, portfolio_greeks)
w2, w3 = int(w[0][0]), int(w[1][0])
print('w2=%s, w3=%s' % (w2,w3))

w2=700, w3=509


In [96]:
# Greeks including delta
portfolio_greeks = [[option1.delta * w1], [option1.gamma * w1], [option1.vega * w1]]
greeks = np.array([[option2.delta, option3.delta], [option2.gamma, option3.gamma], [option2.vega, option3.vega]])
ws = (np.round(np.dot(np.round(greeks, 2), w) + portfolio_greeks))[0][0]
print('ws=%s' % ws)

ws=-40.0


In [99]:
print('Final asset allocation: option1: {}, option2: {}, option3: {}, underlying asset: {}'.format(w1,w2,w3,ws))
price = w1*option1.price, w2*option2.price, w3*option3.price, ws*stock.price
print('price option1: %.2f, option2: %.2f, option3: %.2f, stock: %.2f' %  price)
print('total price:', sum(price))
print('origin delta: %.2f, gamma: %.2f, vega: %.2f, theta: %.2f' %(w1*option1.delta, w1*option1.gamma, w1*option1.vega, w1*option1.theta))
print('target delta: %.2f, gamma: %2.f, vega: %.2f, theta: %.2f' % (
    w1*option1.delta+w2*option2.delta+w3*option3.delta-ws*stock.delta, w1*option1.gamma+w2*option2.gamma+w3*option3.gamma-ws*stock.gamma,
    w1*option1.vega+w2*option2.vega+w3*option3.vega-ws*stock.vega, w1*option1.theta+w2*option2.theta+w3*option3.theta-ws*stock.theta
))

Final asset allocation: option1: -1000, option2: 700, option3: 509, underlying asset: -40.0
price option1: -8220.00, option2: 4242.00, option3: 2265.05, stock: -18880.00
total price: -20592.95
origin delta: -293.00, gamma: -7.00, vega: -410.00, theta: 343.00
target delta: -3.12, gamma: -0, vega: -1.23, theta: -3.35


In [48]:
a = [[1, 2], [3, 4]]             # 原始矩阵
inv = np.linalg.inv(a)           # 求逆矩阵
identity = np.dot(inv, a)        # 验证逆矩阵，预期得到单位矩阵
a, inv, np.round(identity, 2)

([[1, 2], [3, 4]],
 array([[-2. ,  1. ],
        [ 1.5, -0.5]]),
 array([[1., 0.],
        [0., 1.]]))

In [49]:
b = [[5],[6]]       # 目标矩阵
w = np.dot(inv, b)  # 用逆矩阵得到权重
w, np.dot(a,w)-b    # 验证权重，预期得到零矩阵

(array([[-4. ],
        [ 4.5]]),
 array([[0.],
        [0.]]))

In [51]:
c = [[-7],[-5],[-6]]
d = [[8, 9],[1, 2],[3, 4]]
e = np.dot(d, w)
e, e+c

(array([[8.5],
        [5. ],
        [6. ]]),
 array([[1.5],
        [0. ],
        [0. ]]))

In [52]:
w1 = w[0][0]
w2 = w[1][0]
w3 = (e+c)[0][0]
w1, w2, w3

(-4.0, 4.5, 1.5)

In [56]:
w1*d[1][0]+w2*d[1][1]+c[1]

array([0.])

In [57]:
w1*d[2][0]+w2*d[2][1]+c[2]

array([0.])

In [64]:
w1*d[0][0]+w2*d[0][1]+c[0]-[w3]

array([0.])